In [93]:
import asyncio
import websockets
import requests
import json
import pandas as pd
import nest_asyncio
from datetime import datetime as dt
from datetime import timedelta
from datetime import date
import time
import numpy as np
from dash import Dash, dcc, html, Input, Output
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
nest_asyncio.apply()

#Time-related functions
def datetime_to_timestamp(datetime): return int(dt.timestamp(datetime)*1000)
def timestamp_to_datetime(timestamp): return dt.fromtimestamp(timestamp/1000)
def date_to_timestamp(date): return datetime_to_timestamp(dt.combine(date, dt.min.time()))
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

#Gathering and manipulating the (option + currency) crypto data        
async def call_api(msg):
    async with websockets.connect('wss://www.deribit.com/ws/api/v2') as websocket:
        await websocket.send(msg)
        while websocket.open:
            response = await websocket.recv()
            json_par = json.loads(response)
            return(json_par)
                    
class MarketData:
    
    def __init__(self, currency, start_date, end_date, interval = 3600):
        self.currency = currency
        self.start_date = start_date
        self.end_date = end_date
        self.interval = interval

    def currency_data(self, limit = 1000):
        currency_pair = self.currency.lower()+"usd"
        has_more = True 
        price_list = []
        params = {"start":int(date_to_timestamp(self.start_date)/1000), "step":self.interval, "limit":limit}
        while has_more == True:
            data = requests.get(f"https://www.bitstamp.net/api/v2/ohlc/{currency_pair}/", params = params).json()['data']['ohlc']
            price_list.extend(data)
            if int(data[-1]["timestamp"]) > int(date_to_timestamp(self.end_date)/1000):
                break
            params = {"start":int(data[-1]["timestamp"])+1, "step":self.interval, "limit":limit}
        currency_data = pd.DataFrame(price_list)
        currency_data = currency_data.astype(float)
        currency_data["timestamp"] = currency_data["timestamp"].astype(int)
        currency_data["date_time"] = currency_data["timestamp"].apply(lambda x: dt.fromtimestamp(x))
        currency_data = currency_data[currency_data["timestamp"] <= int(date_to_timestamp(self.end_date)/1000)]
        return currency_data

    def msg(self, kind = "option", count = 1000, include_old = True, sorting = "asc"):
        return {"jsonrpc" : "2.0",
                "id" : 1888,
                "method" : "public/get_last_trades_by_currency_and_time",
                "params" : {"currency": self.currency, 
                            "kind": kind,
                            "start_timestamp": self.updated_start_timestamp,
                            "end_timestamp": date_to_timestamp(self.end_date),
                            "count": count,
                            "include_old": include_old,
                            "sorting": sorting
                           }
                }

    def option_data(self):
        has_more = True 
        trades_list = []
        self.updated_start_timestamp = date_to_timestamp(self.start_date)
        while has_more == True:
            data = asyncio.get_event_loop().run_until_complete(call_api(json.dumps(MarketData.msg(self))))
            trades_list.extend(data["result"]["trades"])
            has_more = data["result"]["has_more"]
            self.updated_start_timestamp = int(data["result"]["trades"][-1]["timestamp"]+1)

        option_data = pd.DataFrame(trades_list)
        option_data = option_data[["timestamp", "price", "instrument_name", "index_price", "direction", "amount", 'iv']]
        option_data["kind"] = option_data["instrument_name"].apply(lambda x: str(x).split("-")[0])
        option_data["maturity_date"] = option_data["instrument_name"].apply(lambda x: str(x).split("-")[1])
        option_data["maturity_date"] = option_data["maturity_date"].apply(lambda x: dt.strptime(x, "%d%b%y"))
        option_data["strike_price"] = option_data["instrument_name"].apply(lambda x: int(str(x).split("-")[2]))
        option_data["moneyness"] = option_data["index_price"]/option_data["strike_price"]
        option_data["option_type"] = option_data["instrument_name"].apply(lambda x: str(x).split("-")[3])
        option_data["price"] = (option_data["price"]*option_data["index_price"]).apply(lambda x: round(x,2))
        option_data["date_time"] = option_data["timestamp"].apply(timestamp_to_datetime)
        option_data["time_to_maturity"] = option_data['maturity_date'] - option_data["date_time"]
        option_data["time_to_maturity"] = option_data["time_to_maturity"].apply(lambda x: max(round(x.total_seconds()/31536000,3),1e-04))
        option_data['option_type'] = option_data['option_type'].apply(lambda x: str(x).lower())
        option_data["iv"] = round(option_data["iv"]/100,3)
        option_data["time_to_maturity"] = option_data["time_to_maturity"]*365
        option_data.drop(['timestamp'], axis=1, inplace = True)
        return option_data[['date_time','price', 'index_price', 'direction', 'amount', 'kind', 'time_to_maturity','strike_price', 'moneyness' ,'option_type', 'iv', 'maturity_date', 'instrument_name']]

#Figures
def update_2d_iv_graph(option_data, date, poly_plot = False, points = True, order = 2, lower_bound = 0.8, upper_bound = 1.2):
    data = option_data[option_data["date_time"].apply(lambda x: x.date()) == date]
    __data = data[(data["moneyness"]>lower_bound)&(data["moneyness"]<upper_bound)]
    data.reset_index(inplace = True, drop = True)
    fig = go.Figure()
    if points == False and poly_plot == False:
        return(print("Set either poly_plot or points equal to True"))
    if points == True:
        for maturity_date in sorted(set(data["maturity_date"])):
            _data = data[data["maturity_date"] == maturity_date]
            fig.add_trace(go.Scatter(
                x=_data["moneyness"],
                y=_data["iv"],
                mode='markers',
                marker=dict(
                    size=4,   
                    opacity=0.8
                ),
            name = f"{maturity_date.date()}"))
        
    poly = np.polyfit(__data["moneyness"],__data["iv"], order)
    if poly_plot == True:
        fig.add_trace(go.Scatter(
            x = np.linspace(min(__data["moneyness"]),max(__data["moneyness"]), 1000),
            y = np.polyval(poly,np.linspace(min(__data["moneyness"]),max(__data["moneyness"]), 1000)),
        name = f"2e order poly {data.date_time[0].date()}"))
        
    fig.update_layout(
        title = f"{data['kind'][0]} Volatility Smiles of {date}",
        legend_title_text='Maturity dates',
        xaxis_title = "Moneyness",
        yaxis_title = "Implied Volatility")
    return fig

def update_2d_price_graph(price_data):
    fig = go.Figure(data=go.Ohlc(x=price_data['date_time'],
                    open=price_data['open'],
                    high=price_data['high'],
                    low=price_data['low'],
                    close=price_data['close']))
    fig.update_layout(title = f"OHLC from {min(price_data['date_time']).date()} to {min(price_data['date_time']).date()}")
    return fig

def update_3d_graph(option_data):
    fig = go.Figure(
        data = [go.Scatter3d(
            x=option_data["moneyness"],
            y=option_data["time_to_maturity"],
            z=option_data["iv"],
            name = "instrument_name",
            text = option_data["instrument_name"],
            customdata = option_data["date_time"],
            hovertemplate=
                "<b>%{text}</b><br><br>" +
                "Implied volatility: %{z:.3f}<br>" +
                "Time to maturity: %{y:.0f} days <br>" +
                "Moneyness: %{x:.3f}<br>" +
                "Date: %{customdata}<br>"+
                "<extra></extra>",
            mode='markers',
            marker=dict(
                size=4,
                color=option_data["iv"],                
                colorscale='Viridis',   
                opacity=0.8
            ))])

    fig.update_layout(
        scene = dict(
            xaxis_title = "Moneyness",
            yaxis_title = "Time to maturity (days)",
            zaxis_title = "Implied Volatility"),
        margin =dict(l=0, r=0, b=0, t=0), 
        hoverlabel = dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell")
    )

    return fig

In [102]:
#Inputs
currency = "BTC"
begin_date = date(2022, 12, 26)
end_date = dt.today().date()

update_2d_price_graph(MarketData(currency,begin_date,end_date,300).currency_data())
#update_2d_price_graph(MarketData(currency,begin_date,end_date,300).currency_data()).write_html("ohlc.html")

In [103]:
#Inputs
currency = "BTC"
begin_date = date(2022, 12, 26)
end_date = dt.today().date()

update_2d_iv_graph(option_data = MarketData(currency,begin_date,end_date).option_data(), date = begin_date, poly_plot = False, points = True)
#update_2d_iv_graph(option_data = MarketData(currency,begin_date,end_date).option_data(), date = begin_date, poly_plot = False, points = True).write_html("implied_volatility_smile.html")

In [104]:
#Inputs
currency = "BTC"
begin_date = date(2022, 12, 26)
end_date = dt.today().date()

update_3d_graph(option_data = MarketData(currency,begin_date,end_date).option_data())
#update_3d_graph(option_data = MarketData(currency,begin_date,end_date).option_data()).write_html("implied_volatility_surface.html")